# Indexing

## Direct Indexing

## Example Data from geonames

From http://download.geonames.org/export/dump/

```
The main 'geoname' table has the following fields :
---------------------------------------------------
geonameid         : integer id of record in geonames database
name              : name of geographical point (utf8) varchar(200)
asciiname         : name of geographical point in plain ascii characters, varchar(200)
alternatenames    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)
latitude          : latitude in decimal degrees (wgs84)
longitude         : longitude in decimal degrees (wgs84)
feature class     : see http://www.geonames.org/export/codes.html, char(1)
feature code      : see http://www.geonames.org/export/codes.html, varchar(10)
country code      : ISO-3166 2-letter country code, 2 characters
cc2               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters
admin1 code       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)
admin2 code       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80) 
admin3 code       : code for third level administrative division, varchar(20)
admin4 code       : code for fourth level administrative division, varchar(20)
population        : bigint (8 byte int) 
elevation         : in meters, integer
dem               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.
timezone          : the iana timezone id (see file timeZone.txt) varchar(40)
modification date : date of last modification in yyyy-MM-dd format
```



In [3]:
# Download
#!curl http://download.geonames.org/export/dump/allCountries.zip --output data/allCountries.zip


In [4]:

def read_geonames(filename):
    from io import TextIOWrapper
    import zipfile
    import csv
    result = {}
    with zipfile.ZipFile(f'data/{filename}.zip') as myzip:
        with myzip.open(f'{filename}.txt', 'r') as csv_file:
            reader = csv.DictReader(TextIOWrapper(csv_file, 'utf-8'), delimiter='\t', fieldnames=['geonameid', 'name', 'asciiname', 'alternatenames', 'latitude', 'longitude', 'feature class', 'feature code', 'country code', 'cc2', 'admin1 code', 'admin2 code', 'admin3 code', 'admin4 code', 'population', 'elevation', 'dem', 'timezone', 'modification date'])
            for data in reader:
                result[int(data['geonameid'])] = data

    return result

cities = read_geonames('cities500')  # Use this if you don't want to wait too long...
places = read_geonames('allCountries')

## Visualize

Print the size and look at the first entry. Refrain from using Pandas as we want to code retrieval ourselves.

In [5]:
print(len(places))
next(iter(places.values()))

12410889


{'geonameid': '2986043',
 'name': 'Pic de Font Blanca',
 'asciiname': 'Pic de Font Blanca',
 'alternatenames': 'Pic de Font Blanca,Pic du Port',
 'latitude': '42.64991',
 'longitude': '1.53335',
 'feature class': 'T',
 'feature code': 'PK',
 'country code': 'AD',
 'cc2': '',
 'admin1 code': '00',
 'admin2 code': '',
 'admin3 code': '',
 'admin4 code': '',
 'population': '0',
 'elevation': '',
 'dem': '2860',
 'timezone': 'Europe/Andorra',
 'modification date': '2014-11-05'}

## Lookup
Let's find London in the data - oh, we have more than one?

In [6]:
import time

def lin_search(db, attribute, query):
    """Linear search through db, looking for entities with an attribute equal to query."""
    for town in db.values():
        if town[attribute] == query:
            yield town

start = time.time()
for result in lin_search(places, "name", "London"): print(result)
elapsed = time.time() - start
print(f"took {elapsed:.2g}s")

{'geonameid': '3581797', 'name': 'London', 'asciiname': 'London', 'alternatenames': '', 'latitude': '17.98333', 'longitude': '-88.43333', 'feature class': 'P', 'feature code': 'PPL', 'country code': 'BZ', 'cc2': '', 'admin1 code': '04', 'admin2 code': '', 'admin3 code': '', 'admin4 code': '', 'population': '0', 'elevation': '', 'dem': '15', 'timezone': 'America/Belize', 'modification date': '1993-12-18'}
{'geonameid': '6058559', 'name': 'London', 'asciiname': 'London', 'alternatenames': '', 'latitude': '43.08339', 'longitude': '-81.29975', 'feature class': 'L', 'feature code': 'AREA', 'country code': 'CA', 'cc2': '', 'admin1 code': '08', 'admin2 code': '', 'admin3 code': '', 'admin4 code': '', 'population': '0', 'elevation': '', 'dem': '281', 'timezone': 'America/Toronto', 'modification date': '2006-01-18'}
{'geonameid': '6058560', 'name': 'London', 'asciiname': 'London', 'alternatenames': 'Landona,London,Londonas,Londono,YXU,leondeon,lndn,lndn  antaryw,londoni,lun dui,lun dun,lwndwn,r

### Direct Lookup

When reading the data, we conveniently recorded a dictionary with the `geonameid` as key. If we know it, lookup is quick:

In [7]:
import time
start = time.time()
print(places[2643743])
elapsed = time.time() - start
print(f"took {elapsed:.2g}s")

{'geonameid': '2643743', 'name': 'London', 'asciiname': 'London', 'alternatenames': 'ILondon,LON,Lakana,Landan,Landen,Ljondan,Llundain,Lodoni,Londain,Londan,Londar,Londe,Londen,Londin,Londinium,Londino,Londn,London,London osh,Londona,Londonas,Londoni,Londono,Londons,Londonu,Londra,Londres,Londrez,Londri,Londro,Londye,Londyn,Londýn,Lonn,Lontoo,Loundres,Luan GJon,Lun-tun,Lunden,Lundra,Lundun,Lundunir,Lundúnir,Lung-dung,Lunnainn,Lunnin,Lunnon,Luân Đôn,Lùn-tûn,Lùng-dŭng,Lûn-tun,Lākana,Lůndůn,Lọndọnu,Ranana,Rānana,ilantan,ladana,landan,landana,leondeon,lndn,london,londoni,lun dui,lun dun,lwndwn,lxndxn,rondon,Łondra,Λονδίνο,Лондан,Лондон,Лондон ош,Лондонъ,Лёндан,Լոնդոն,לאנדאן,לונדון,لأندأن,لندن,لوندون,لەندەن,ܠܘܢܕܘܢ,लंडन,लंदन,लण्डन,लन्डन्,लन्दन,লন্ডন,ਲੰਡਨ,લંડન,ଲଣ୍ଡନ,இலண்டன்,లండన్,ಲಂಡನ್,ലണ്ടൻ,ලන්ඩන්,ลอนดอน,ລອນດອນ,ལོན་ཊོན།,လန်ဒန်မြို့,ლონდონი,ለንደን,ᎫᎴ ᏗᏍᎪᏂᎯᏱ,ロンドン,伦敦,倫敦,런던', 'latitude': '51.50853', 'longitude': '-0.12574', 'feature class': 'P', 'feature code': 'PPLC', 'country code': 'GB', 'cc2':

### Secondary Index

We want fast lookups for place name, so we create a secondary index: a dictionary that maps from place name to the primary, unique key (geonameid). The dictionary values are sets of ids. We also convert geonameids into numbers to save space.

Index creation takes a long time, but lookup is ~1000x faster than with linear search.

In [8]:
name_idx = {}
for town in places.values():
    id = int(town['geonameid'])
    # matching_places = None
    # if town['name'] in name_idx:
    #     matching_places = name_idx[town['name']]
    # else:
    #     matching_places = set()
    #     name_idx[town['name']] = matching_places
    # matching_places.add(id)

    # setdefault retrieves the value for key or adds
    # the given default value if the key does not exist.
    name_idx.setdefault(town['name'], set()).add(id)

def index_search(db, idx, query):
    """Search using the secondary index 'idx', then look up directly in the database."""
    for id in idx[query]:
        yield db[id]

start = time.time()
for result in index_search(places, name_idx, "London"): print(result)
elapsed = time.time() - start
print(f"took {elapsed:.2g}s")


{'geonameid': '1705729', 'name': 'London', 'asciiname': 'London', 'alternatenames': '', 'latitude': '6.00972', 'longitude': '125.12944', 'feature class': 'P', 'feature code': 'PPL', 'country code': 'PH', 'cc2': '', 'admin1 code': '12', 'admin2 code': '70', 'admin3 code': '126303000', 'admin4 code': '', 'population': '0', 'elevation': '', 'dem': '14', 'timezone': 'Asia/Manila', 'modification date': '2017-12-13'}
{'geonameid': '5367815', 'name': 'London', 'asciiname': 'London', 'alternatenames': 'London,Londres,New London', 'latitude': '36.47606', 'longitude': '-119.44318', 'feature class': 'P', 'feature code': 'PPL', 'country code': 'US', 'cc2': '', 'admin1 code': 'CA', 'admin2 code': '107', 'admin3 code': '', 'admin4 code': '', 'population': '1869', 'elevation': '91', 'dem': '93', 'timezone': 'America/Los_Angeles', 'modification date': '2011-05-14'}
{'geonameid': '8610441', 'name': 'London', 'asciiname': 'London', 'alternatenames': '', 'latitude': '14.32737', 'longitude': '120.97552', 

## Range Queries

We want to find all towns with a population between 10M and 15M, but avoid a costly linear pass.

Solution: use a lists of (population, geonameid) tuples, sorted by population. Using binary search, we can find the boundaries for the requested range.

In [10]:
# Create the index.
population_idx = []
for town in places.values():
    # Only include the 'populated place' class, but exclude admin areas and countries.
    if town['feature class'] == 'P':
        population_idx.append((int(town['population']), int(town['geonameid'])))
population_idx.sort()  # Sort first by population, then id

# Use bisect instead of implementing binary search ourselves:
import bisect
# Use operator.itemgetter(0) to extract the population from the (pop, id) tuple.
import operator
import time
start = time.time()

# The index of the smallest entity in population_idx greater or equal than 10M.
lower = bisect.bisect_left(population_idx, 10e6, key=operator.itemgetter(0))
# The index of the largest entity in population_idx greater than 15M.
upper = bisect.bisect_right(population_idx, 15e6, key=operator.itemgetter(0))
print(f"Found {upper - lower} entities.")
for idx in range(lower, upper):
    entity = places[population_idx[idx][1]]
    print(entity['name'], entity['population'])

elapsed = time.time() - start
print(f"took {elapsed:.2g}s")

Found 12 entities.
Seoul 10349312
Dhaka 10356500
Moscow 10381222
Wuhan 10392693
Delhi 10927986
Tianjin 11090314
Karachi 11624219
Mexico City 12294193
São Paulo 12400232
Mumbai 12691836
Chengdu 13568357
Istanbul 14804116
took 0.025s


## Trees

Instead of binary search on a sorted array, we can achieve the same speed (albeit using more memory) by organizing the population index as a [binary search tree](https://en.wikipedia.org/wiki/Binary_search_tree): Each node stores two (hence, _binary_) child nodes, where all nodes on the left have a lower population, and all nodes on the right have a equal or greater population. Ideally, the root node represents the median population of all cities, so that each subtree contains the same number of entities.

[![Example Graph](figures/Binary%20Tree%20by%20Population.svg)](http://magjac.com/graphviz-visual-editor/?dot=digraph%20BST_Pop%20%7B%0A%20%20%20%20rankdir%3DTB%3B%20ordering%3Dout%3B%0A%09node%20%5Bshape%20%3D%20circle%3B%20width%3D1%3B%20fixedwidth%3Dtrue%5D%3B%0A%0A%20%20%20%20Seoul%5Blabel%3D%3Cpop%3A%2010349312%3CBR%2F%3E%5CN%3E%5D%0A%20%20%20%20Moscow%5Blabel%3D%3Cpop%3A%2010381222%3CBR%2F%3E%5CN%3E%5D%0A%20%20%20%20Wuhan%5Blabel%3D%3Cpop%3A%2010392693%3CBR%2F%3E%5CN%3E%5D%0A%20%20%20%20Tianjin%5Blabel%3D%3Cpop%3A%2011090314%3CBR%2F%3E%5CN%3E%5D%0A%20%20%20%20Karachi%5Blabel%3D%3Cpop%3A%2011624219%3CBR%2F%3E%5CN%3E%5D%0A%20%20%20%20S%C3%A3o_Paulo%5Blabel%3D%3Cpop%3A%2012400232%3CBR%2F%3ES%C3%A3o%20Paulo%3E%5D%0A%20%20%20%20Chengdu%5Blabel%3D%3Cpop%3A%2013568357%3CBR%2F%3E%5CN%3E%5D%0A%20%20%20%20%0A%0A%20%20%20%20Tianjin%20-%3E%20Moscow%20%5Blabel%3Dleft%5D%3B%0A%20%20%20%20Tianjin%20-%3E%20S%C3%A3o_Paulo%20%5Blabel%3Dright%5D%3B%0A%20%20%20%20Moscow%20-%3E%20Seoul%20%5Blabel%3Dleft%5D%3B%0A%20%20%20%20Moscow%20-%3E%20Wuhan%20%5Blabel%3Dright%5D%3B%0A%20%20%20%20S%C3%A3o_Paulo%20-%3E%20Karachi%20%5Blabel%3Dleft%5D%3B%0A%20%20%20%20S%C3%A3o_Paulo%20-%3E%20Chengdu%20%5Blabel%3Dright%5D%3B%0A%0A%7D%0A)

We are going to build such a tree below. One crucial part is finding the right nodes such that the tree is approximately balanced and does not degenerate into a _vine_:

![BST Vine](figures/bst-vine.png)

Ideally, we choose the median element from the sorted array as root, and then recursively continue to choose the left and right children from the left and right halves of the array.

In [11]:
class Node:
    """A tree node, also represents an entire (sub-)tree."""
    def __init__(self, key, value):
        self.key = key
        self.value = value
        self.left = None
        self.right = None

def build_tree(sorted_tuples, left=None, right=None):
    """Builds a binary search tree from a sorted tuple list."""
    left = left if left is not None else 0
    right = right if right is not None else len(sorted_tuples) - 1

    # Stop when the given range is empty.
    if left > right:
        return None
    median = (left + right) // 2
    element = sorted_tuples[median]
    node = Node(element[0], element[1])
    node.left = build_tree(sorted_tuples, left, median - 1)
    node.right = build_tree(sorted_tuples, median + 1, right)
    return node

population_tree = build_tree(population_idx)

Now let's query the tree index:

In [13]:
def tree_search(node, lower, upper):
    """Traverses tree from lower to upper key."""
    if node is None:
        return  # reached the bottom of the tree
    if lower < node.key:
        # return elements on our left
        yield from tree_search(node.left, lower, upper)
    if lower <= node.key <= upper:
        yield node.value  # return this node's value
    if node.key <= upper:
        # return elements on our right
        yield from tree_search(node.right, lower, upper)

import time
start = time.time()

for geonameid in tree_search(population_tree, 10e6, 15e6):
    entity = places[geonameid]
    print(entity['name'], entity['population'])

elapsed = time.time() - start
print(f"took {elapsed:.2g}s")

Seoul 10349312
Dhaka 10356500
Moscow 10381222
Wuhan 10392693
Delhi 10927986
Tianjin 11090314
Karachi 11624219
Mexico City 12294193
São Paulo 12400232
Mumbai 12691836
Chengdu 13568357
Istanbul 14804116
took 0.001s
